# Web Scraping for Indeed.com Australia and Salary Prediction

### PART 2: Data Cleaning

This project is a test of three major skills: collecting data by scraping a website, using natural language processing, and building a binary classifier.

Author : Ayesha Khatib

In [536]:
# Import packages

import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import bs4
import urllib
import urllib.request
import re

from bs4 import BeautifulSoup
from time import sleep

%matplotlib inline

## Data Cleaning :

In [537]:
# Read the results of the Website scrapping.

#jobs_df.to_csv('./job_posts_scrapped.csv',index=False, encoding='utf-8')
jobs_df = pd.read_csv('job_posts_scrapped.csv')

jobs_df.shape

(1260, 5)

In [538]:
jobs_df.head()

,Company,Job_Title,Summary,Location,Salary
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000 a year"
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000 a year"
2,None,Junior Data Scientist,Opportunity to start your career i...,Sydney Central Business District NSW,None
3,DataRobot,Customer Facing Data Scientist,Customer Facing Data Scientists wo...,Sydney NSW,None
4,Freshwater Group,Data Scientist,The Data Scientist will:. Manage d...,Sydney NSW,None


In [539]:
# Dropping any duplicate rows:
jobs_df = jobs_df.drop_duplicates()
jobs_df.reset_index(drop=True, inplace=True)
jobs_df.shape

(516, 5)

In [540]:
# Check each column for any discrepencies.

jobs_df.Location.unique()

# Observation : 
# 1. Location columns consists of the major Aus cities, suburbs and respective state names.
# 2. There is one Location entry as 'New South Wales' which is not consistent with the rest of entries. Hence, will 
    #normalize by reaplcing it as 'Sydney, NSW'

array(['Sydney NSW', 'Sydney Central Business District NSW',
       'Chippendale NSW', 'Pyrmont NSW', 'Hills District NSW',
       'Melbourne VIC', 'Parkville VIC', 'Toorak VIC', 'Research VIC',
       'Port Melbourne VIC', 'Melbourne City Centre VIC', 'Canberra ACT',
       'Brisbane QLD', 'St Leonards NSW', 'Sydney Inner Suburbs NSW',
       'Sydney Western Suburbs NSW', 'Westmead NSW', 'Perth WA',
       'Watermans Bay WA', 'Perth Central Business District WA',
       'Northern Beaches NSW', 'New South Wales', 'Newcastle NSW',
       'Hornsby NSW', 'Chatswood NSW', 'Rhodes NSW',
       'Sydney Eastern Suburbs NSW', 'Parramatta NSW', 'Burwood NSW',
       'Mascot NSW', 'Glendenning NSW', 'Camperdown NSW', 'Burwood VIC',
       'Dandenong VIC', 'Yarraville VIC', 'Albion VIC', 'Queanbeyan NSW',
       'Greenway ACT', 'Eight Mile Plains QLD', 'Geebung QLD',
       'Brisbane Central Business District QLD', 'Dutton Park QLD',
       'Redfern NSW', 'North Sydney NSW', 'North Ryde NSW',
   

In [541]:
# Replacing it as 'Sydney, NSW'
jobs_df = jobs_df.replace('New South Wales', 'Sydney NSW')
jobs_df.Location.unique()


array(['Sydney NSW', 'Sydney Central Business District NSW',
       'Chippendale NSW', 'Pyrmont NSW', 'Hills District NSW',
       'Melbourne VIC', 'Parkville VIC', 'Toorak VIC', 'Research VIC',
       'Port Melbourne VIC', 'Melbourne City Centre VIC', 'Canberra ACT',
       'Brisbane QLD', 'St Leonards NSW', 'Sydney Inner Suburbs NSW',
       'Sydney Western Suburbs NSW', 'Westmead NSW', 'Perth WA',
       'Watermans Bay WA', 'Perth Central Business District WA',
       'Northern Beaches NSW', 'Newcastle NSW', 'Hornsby NSW',
       'Chatswood NSW', 'Rhodes NSW', 'Sydney Eastern Suburbs NSW',
       'Parramatta NSW', 'Burwood NSW', 'Mascot NSW', 'Glendenning NSW',
       'Camperdown NSW', 'Burwood VIC', 'Dandenong VIC', 'Yarraville VIC',
       'Albion VIC', 'Queanbeyan NSW', 'Greenway ACT',
       'Eight Mile Plains QLD', 'Geebung QLD',
       'Brisbane Central Business District QLD', 'Dutton Park QLD',
       'Redfern NSW', 'North Sydney NSW', 'North Ryde NSW',
       'Barangaroo NSW

In [542]:
# Creating two new columns -- State and City from Location column.

# Using regular expression to extract strings with 2 or 3 captial letters or return whole string.

# Define the reg exp for state
state_re = '\s[A-Z]{2,3}'
jobs_df['State'] = [re.findall(state_re, str(i)) if len(re.findall(state_re, str(i))) > 0 else i for i in jobs_df['Location']]

In [543]:
#Convert any lists to strings
for i in jobs_df.index:
    if len(jobs_df['State'][i]) < 4:
        jobs_df['State'][i] = jobs_df['State'][i][0]
    else:
        jobs_df['State'][i] = jobs_df['State'][i]

In [544]:
#Removing any spaces
jobs_df['State'] = jobs_df['State'].apply(lambda x: x.replace(' ',''))

In [545]:
jobs_df['State'].value_counts()

NSW    301
VIC     92
QLD     55
ACT     34
WA      34
Name: State, dtype: int64

In [546]:
#Check unique values again
jobs_df['State'].unique()

array(['NSW', 'VIC', 'ACT', 'QLD', 'WA'], dtype=object)

In [547]:
jobs_df.Location.value_counts()

Sydney NSW                                217
Melbourne VIC                              73
Brisbane QLD                               47
Canberra ACT                               33
Perth WA                                   30
Sydney Central Business District NSW       28
Melbourne City Centre VIC                   9
Sydney Western Suburbs NSW                  7
Brisbane Central Business District QLD      4
Rhodes NSW                                  4
Northern Beaches NSW                        3
Perth Central Business District WA          3
Mascot NSW                                  3
Sydney Inner Suburbs NSW                    3
Dandenong VIC                               2
Parramatta NSW                              2
Eight Mile Plains QLD                       2
Hills District NSW                          2
Alexandria NSW                              2
Burwood NSW                                 2
North Sydney NSW                            2
Research VIC                      

In [548]:
# Seperating the City and State in seperate columns.

# Column City/State created for manipulation only and splits the City and State as tuples.
jobs_df['CityState'] = jobs_df['Location'].str.split(' ')


In [549]:
# Extracting only the 0th index element which are the cities.
jobs_df['City']= jobs_df['CityState'].str[0]

jobs_df.head(5)

,Company,Job_Title,Summary,Location,Salary,State,CityState,City
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000 a year",NSW,"[Sydney, NSW]",Sydney
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000 a year",NSW,"[Sydney, NSW]",Sydney
2,None,Junior Data Scientist,Opportunity to start your career i...,Sydney Central Business District NSW,None,NSW,"[Sydney, Central, Business, District, NSW]",Sydney
3,DataRobot,Customer Facing Data Scientist,Customer Facing Data Scientists wo...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney
4,Freshwater Group,Data Scientist,The Data Scientist will:. Manage d...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney


In [551]:
# Verify the City column
jobs_df.City.unique()

array(['Sydney', 'Chippendale', 'Pyrmont', 'Hills District', 'Melbourne',
       'Parkville', 'Toorak', 'Research', 'Port Melbourne', 'Canberra',
       'Brisbane', 'St Leonards', 'Westmead', 'Perth', 'Watermans Bay',
       'Sydneyern', 'Newcastle', 'Hornsby', 'Chatswood', 'Rhodes',
       'Parramatta', 'Burwood', 'Mascot', 'Glendenning', 'Camperdown',
       'Dandenong', 'Yarraville', 'Albion', 'Queanbeyan', 'Greenway',
       'Eight Mile Plains', 'Geebung', 'Dutton Park', 'Redfern',
       'Barangaroo', 'Alexandria', 'Rydalmere', 'Macquarie Park ',
       'Surry Hills', 'Bondi Junction', 'Randwick', 'Crows Nest ',
       'Penrith', 'Muswellbrook'], dtype=object)

In [553]:
# Deleting the column names "CityState"

#del jobs_df['CityState']
jobs_df.head(5)

,Company,Job_Title,Summary,Location,Salary,State,CityState,City
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000 a year",NSW,"[Sydney, NSW]",Sydney
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000 a year",NSW,"[Sydney, NSW]",Sydney
2,None,Junior Data Scientist,Opportunity to start your career i...,Sydney Central Business District NSW,None,NSW,"[Sydney, Central, Business, District, NSW]",Sydney
3,DataRobot,Customer Facing Data Scientist,Customer Facing Data Scientists wo...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney
4,Freshwater Group,Data Scientist,The Data Scientist will:. Manage d...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney


In [554]:
#Inspecting for nulls
jobs_df.isnull().sum()

Company      0
Job_Title    0
Summary      0
Location     0
Salary       0
State        0
CityState    0
City         0
dtype: int64

In [555]:
 # pandas.core.frame.DataFrame
jobs_df.dtypes

Company      object
Job_Title    object
Summary      object
Location     object
Salary       object
State        object
CityState    object
City         object
dtype: object

In [556]:
# Keeping the Null Salary column values and translating them as 'No info'

jobs_df['Salary'] = jobs_df['Salary'].apply(lambda x: np.nan if x == 'No info' else x)

In [557]:
# Only 86 jobs posted have Salary data but we cannot remove more than 1000 entries with no Salary data.
len(jobs_df.Salary.unique())

86

In [558]:
# Check the unique values of Salary column
list(jobs_df['Salary'].unique()[:20])

['                $70,000 - $90,000 a year',
 '                $80,000 - $100,000 a year',
 'None',
 '                $65,000 - $110,000 a year',
 '                $160,000 - $170,000 a year',
 '                $50 an hour',
 '                $70,000 - $80,000 a year',
 '                $100,000 - $145,000 a year',
 '                $100,000 - $150,000 a year',
 '                $150,000 a year',
 '                $46,000 - $60,000 a year',
 '                $61,784 - $84,816 a year',
 '                $120 - $150 an hour',
 '                $110 - $120 an hour',
 '                $87,257 - $91,942 a year',
 '                $74,663 - $90,531 a year',
 '                $76,615 - $82,202 a year',
 '                $97,000 - $105,000 a year',
 '                $93,000 a year',
 '                $82,000 - $93,000 a year']

In [559]:
# Defining variables to create regular variables to extract the low and high numbers from the Salary values and return it in two
# new columns.

low_re = '\$\d\S+\s\-'
high_re = '\-\s\$\d\S+\s'
other_re = '[^-]\s\$\d\S+\s(?!-)'

jobs_df['Low_Salary'] = [re.findall(low_re, str(i)) if len(re.findall(low_re, str(i))) > 0 else re.findall(other_re, str(i)) for i in jobs_df['Salary']]
jobs_df['Hi_Salary'] = [re.findall(high_re, str(i)) if len(re.findall(high_re, str(i))) > 0 else re.findall(other_re, str(i)) for i in jobs_df['Salary']]

In [560]:
jobs_df.head()

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000 a year",NSW,"[Sydney, NSW]",Sydney,"[$70,000 -]","[- $90,000 ]"
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000 a year",NSW,"[Sydney, NSW]",Sydney,"[$80,000 -]","[- $100,000 ]"
2,None,Junior Data Scientist,Opportunity to start your career i...,Sydney Central Business District NSW,None,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,[],[]
3,DataRobot,Customer Facing Data Scientist,Customer Facing Data Scientists wo...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,[],[]
4,Freshwater Group,Data Scientist,The Data Scientist will:. Manage d...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,[],[]


In [561]:
# Data frame cleaning by removing unwanted characters and 

unwanted = '$,- '
remove_chars = str.maketrans('','',unwanted)

for i in range(len(jobs_df['Low_Salary'])):
    try:
        jobs_df['Low_Salary'][i] = jobs_df['Low_Salary'][i][0].translate(remove_chars)
    except:
        jobs_df['Low_Salary'][i] = np.nan
for i in range(len(jobs_df['Hi_Salary'])):
    try:
        jobs_df['Hi_Salary'][i] = jobs_df['Hi_Salary'][i][0].translate(remove_chars)
    except:
        jobs_df['Hi_Salary'][i] = np.nan
     

In [562]:
jobs_df.head()

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000 a year",NSW,"[Sydney, NSW]",Sydney,70000,90000
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000 a year",NSW,"[Sydney, NSW]",Sydney,80000,100000
2,None,Junior Data Scientist,Opportunity to start your career i...,Sydney Central Business District NSW,None,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,NaN,NaN
3,DataRobot,Customer Facing Data Scientist,Customer Facing Data Scientists wo...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,NaN,NaN
4,Freshwater Group,Data Scientist,The Data Scientist will:. Manage d...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,NaN,NaN


In [563]:
jobs_df.head()

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000 a year",NSW,"[Sydney, NSW]",Sydney,70000,90000
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000 a year",NSW,"[Sydney, NSW]",Sydney,80000,100000
2,None,Junior Data Scientist,Opportunity to start your career i...,Sydney Central Business District NSW,None,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,NaN,NaN
3,DataRobot,Customer Facing Data Scientist,Customer Facing Data Scientists wo...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,NaN,NaN
4,Freshwater Group,Data Scientist,The Data Scientist will:. Manage d...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,NaN,NaN


In [564]:
# converting the column values to float.

jobs_df['Low_Salary'] = jobs_df['Low_Salary'].astype('float')
jobs_df['Hi_Salary'] = jobs_df['Hi_Salary'].astype('float')

In [565]:
jobs_df.shape 

(516, 10)

In [566]:
# Saving the csv file ---Indeed website data.

jobs_df.to_csv('./data_cleanup_stage1.csv', index=False, encoding='utf-8')

In [567]:
# Creating another data frame.
jobs_df2 = pd.DataFrame()
jobs_df2 = jobs_df.copy()
print(jobs_df2.shape)

(516, 10)


In [568]:
jobs_df2.head()

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000 a year",NSW,"[Sydney, NSW]",Sydney,70000.0,90000.0
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000 a year",NSW,"[Sydney, NSW]",Sydney,80000.0,100000.0
2,None,Junior Data Scientist,Opportunity to start your career i...,Sydney Central Business District NSW,None,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,NaN,NaN
3,DataRobot,Customer Facing Data Scientist,Customer Facing Data Scientists wo...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,NaN,NaN
4,Freshwater Group,Data Scientist,The Data Scientist will:. Manage d...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,NaN,NaN


### Converting the different Salary rates -- monthly,weekly, daily and hourly into yearly rates.

In [569]:
#if the salary contains information on time period, save that time period 
#string in the og_salary_period column

jobs_df2.ix[jobs_df2["Salary"].str.contains("year"), "Salary_period"] = "year"
jobs_df2.ix[jobs_df2["Salary"].str.contains("month"), "Salary_period"] = "month"
jobs_df2.ix[jobs_df2["Salary"].str.contains("week"), "Salary_period"] = "week"
jobs_df2.ix[jobs_df2["Salary"].str.contains("day"), "Salary_period"] = "day"
jobs_df2.ix[jobs_df2["Salary"].str.contains("hour"), "Salary_period"] = "hour"

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.l

In [570]:
# Some of the salary information is listed by month
jobs_df2[jobs_df2['Salary'].str.contains('month')==True]

# No salary with monthly pay posted

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary,Salary_period


In [571]:
jobs_df2.head()

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary,Salary_period
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000 a year",NSW,"[Sydney, NSW]",Sydney,70000.0,90000.0,year
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000 a year",NSW,"[Sydney, NSW]",Sydney,80000.0,100000.0,year
2,None,Junior Data Scientist,Opportunity to start your career i...,Sydney Central Business District NSW,None,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,NaN,NaN,NaN
3,DataRobot,Customer Facing Data Scientist,Customer Facing Data Scientists wo...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,NaN,NaN,NaN
4,Freshwater Group,Data Scientist,The Data Scientist will:. Manage d...,Sydney NSW,None,NSW,"[Sydney, NSW]",Sydney,NaN,NaN,NaN


In [572]:
#We'll convert these to monthly rates by multiplying by 12 to get in terms of annual sal package.
mask = jobs_df[jobs_df['Salary'].str.contains('month')==True].index
jobs_df2['Low_Salary'][mask] = jobs_df2['Low_Salary'][mask].apply(lambda x: x * 12)
jobs_df2['Hi_Salary'][mask] = jobs_df2['Hi_Salary'][mask].apply(lambda x: x * 12)


D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [573]:
# Check if there are any weekly rates salaries in our scrapped data to get in terms of annual sal package..

jobs_df2[jobs_df2['Salary'].str.contains('week')==True]

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary,Salary_period
307,Queanbeyan-Palerang Regional Council,Asset Data Analyst (Ongoing),Asset Data Analyst. Asset Data Ana...,Queanbeyan NSW,"$1,522 - $1,682 a week",NSW,"[Queanbeyan, NSW]",Queanbeyan,1522.0,1682.0,week
390,Ethos BeathChapman,Business Analyst,Senior Business Analyst. At least ...,Sydney NSW,$700 - $800 a week,NSW,"[Sydney, NSW]",Sydney,700.0,800.0,week
484,Bloc Technology,Reporting Analyst / Technical Analyst,Reporting Analyst / Technical BA. ...,Sydney NSW,$700 - $800 a week,NSW,"[Sydney, NSW]",Sydney,700.0,800.0,week


In [574]:
#We'll just convert these to something annual by multiplying by 52

mask = jobs_df2[jobs_df2['Salary'].str.contains('week')==True].index
jobs_df2['Low_Salary'][mask] = jobs_df2['Low_Salary'][mask].apply(lambda x: x * 52)
jobs_df2['Hi_Salary'][mask] = jobs_df2['Hi_Salary'][mask].apply(lambda x: x * 52)


D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [575]:
# Check if there are any daily rates salaries in our scrapped data.

jobs_df2[jobs_df2['Salary'].str.contains('day')==True]

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary,Salary_period
200,Bluefin Resources,Project Analyst,"Excel Pivots, data extraction, tra...",Hills District District NSW,$350 a day,NSW,"[Hills, District, NSW]",Hills District,350.0,350.0,day
229,Hays,Business Analyst - Financial Services,You should have strong stakeholder...,Sydney Central Business District NSW,$780 - $800 a day,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,780.0,800.0,day
317,Nspire Recruitment,Business Analyst – Asset Management (6 mths AC...,5+ years experience in working as ...,Canberra ACT,$850 - $900 a day,ACT,"[Canberra, ACT]",Canberra,850.0,900.0,day
346,Hudson,Automation Tester - Multiple positions,"Identify, prepare and/or manipulat...",Brisbane QLD,$800 - $850 a day,QLD,"[Brisbane, QLD]",Brisbane,800.0,850.0,day
356,Hudson,Integration Solution Architect | LIS Program -...,"Work with the Solution Owner, Seni...",Brisbane QLD,"$1,000 a day",QLD,"[Brisbane, QLD]",Brisbane,1000.0,1000.0,day
458,Endeavour Drinks Group,Delivery Manager- D365,Microsoft Dynamics 365 for operati...,Sydney NSW,$125 - $132 a day,NSW,"[Sydney, NSW]",Sydney,125.0,132.0,day
469,Path4 Technology,Data Analyst - Government Client,Data Analyst – Short Contract - Government Pro...,Sydney NSW,$500 a day,NSW,"[Sydney, NSW]",Sydney,500.0,500.0,day
510,Hays,Senior Business Analyst,Data analysis and interpretation/conversion. E...,Sydney Central Business District NSW,$600 - $850 a day,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,600.0,850.0,day
511,Hays,Business Analyst - Finance,Experience within Business Process...,Sydney Central Business District NSW,$800 - $900 a day,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,800.0,900.0,day


In [576]:
#We'll convert these to an annual rate by multiplying by the number of working days in a year. 
# Working days will be 5 * 52 = 260 days out of 365 days

working_days = 5 * 52 # (5 days * 52 weeks)

mask = jobs_df[jobs_df2['Salary'].str.contains('day')==True].index

jobs_df2['Low_Salary'][mask] = jobs_df2['Low_Salary'][mask].apply(lambda x : x * working_days)
jobs_df2['Hi_Salary'][mask] = jobs_df2['Hi_Salary'][mask].apply(lambda x :  x* working_days)


D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [577]:
#Some per hour
jobs_df2[jobs_df2['Salary'].str.contains('hour')==True]

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary,Salary_period
20,Quay Appointments,St Leonardsatistical Modeller,Both skills and experience with da...,Chippendale NSW,$50 an hour,NSW,"[Chippendale, NSW]",Chippendale,50.0,50.00,hour
74,Talent International,Data Scientist,Perform data wrangling and create ...,Canberra ACT,$120 - $150 an hour,ACT,"[Canberra, ACT]",Canberra,120.0,150.00,hour
78,Hudson,Data Scientist,Hudson has an exciting opportunity...,Canberra ACT,$110 - $120 an hour,ACT,"[Canberra, ACT]",Canberra,110.0,120.00,hour
115,Western Sydney Local Health District,Huntington's Disease Trial Psychologist,It includes all of the duties invo...,Sydney Western Suburbs NSW,$33.40 - $49.65 an hour,NSW,"[Sydney, Western, Suburbs, NSW]",Sydney,33.4,49.65,hour
165,Reo Group,Data Analyst,Reporting to the Commercial Manage...,Sydney Western Suburbs NSW,$35 an hour,NSW,"[Sydney, Western, Suburbs, NSW]",Sydney,35.0,35.00,hour
221,Reo Group,Financial Modelling Analyst,You are an experienced analyst wit...,Sydneyern Beaches NSW,$60 an hour,NSW,"[Northern, Beaches, NSW]",Sydneyern,60.0,60.00,hour
239,SB Recruitment,Corporate Finance – Business Analyst,"Working from project initiation, s...",Sydney Central Business District NSW,$85 an hour,NSW,"[Sydney, Central, Business, District, NSW]",Sydney,85.0,85.00,hour
240,Chandler Macleod,Customer Service Lead,Manage customer service and progra...,Sydney NSW,$50 - $58 an hour,NSW,"[Sydney, NSW]",Sydney,50.0,58.00,hour
250,Project Professional Services,Reporting Analyst,Create a new dashboard from contin...,Melbourne VIC,$50 - $60 an hour,VIC,"[Melbourne, VIC]",Melbourne,50.0,60.00,hour
364,EST10,Business/Data Analyst,Extract business data. Identify da...,Sydney NSW,$50 - $65 an hour,NSW,"[Sydney, NSW]",Sydney,50.0,65.00,hour


In [578]:
#These ones we'll convert to an annual rate by assuming 8 hours a day, for the same number of annual working days as above

mask = jobs_df2[jobs_df['Salary'].str.contains('hour')==True].index
jobs_df2['Low_Salary'][mask] = jobs_df2['Low_Salary'][mask].apply(lambda x: x * working_days * 8)
jobs_df2['Hi_Salary'][mask] = jobs_df2['Hi_Salary'][mask].apply(lambda x: x * working_days * 8)


D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [579]:
#Since we have a range of salary values for many of the jobs, I'll summarize these two range values in an extra column
#The mean salary
jobs_df2['Salary_mean'] = (jobs_df2['Low_Salary'] + jobs_df2['Hi_Salary'])/2

In [580]:
jobs_df2.describe()

,Low_Salary,Hi_Salary,Salary_mean
count,104.000000,104.000000,104.000000
mean,103733.163462,117485.653846,110609.408654
std,44653.524692,48356.641265,46006.577181
min,32500.000000,34320.000000,33410.000000
25%,76181.250000,90000.000000,83127.250000
50%,95275.000000,110000.000000,103033.500000
75%,120000.000000,140000.000000,127980.500000
max,260000.000000,312000.000000,280800.000000


In [581]:
jobs_df2.Salary.replace(regex=True, inplace=True, to_replace=" a year", value=" ") #getting rid 'a year' salary
jobs_df2.head(2)

,Company,Job_Title,Summary,Location,Salary,State,CityState,City,Low_Salary,Hi_Salary,Salary_period,Salary_mean
0,The Eclair Group,Junior Data Scientist,Industry experience as a Data Anal...,Sydney NSW,"$70,000 - $90,000",NSW,"[Sydney, NSW]",Sydney,70000.0,90000.0,year,80000.0
1,Intellify,Junior data scientist/machine learning engineer,We also believe great data science...,Sydney NSW,"$80,000 - $100,000",NSW,"[Sydney, NSW]",Sydney,80000.0,100000.0,year,90000.0


In [782]:
# Saving the file after the usual data clean-up:

#jobs_df2.to_csv('./job_posts_clean_complete.csv',index=False, encoding='utf-8')
#jobs_data.to_csv('./job_posts_clean_complete_new.csv',index=False, encoding='utf-8')

In [745]:
trial = pd.read_csv('cleaned_jobs.csv')
#trial.head()

### Next will be Data cleaning on the words in the three focus columns : 'Location', 'Job_Title' and 'Summary'

In [783]:
# Reading the data cleaned up file
jobs = pd.read_csv('job_posts_clean_complete_new.csv')

jobs.head()

,Company,Job_Title,Summary,Location,Salary,State,City,Low_Salary,Hi_Salary,Salary_period,...,Job_Title_finance,Job_Title_junior,Job_Title_financial,Job_Title_pricing,Job_Title_analytics,State_ACT,State_NSW,State_QLD,State_VIC,State_WA
0,The Eclair Group,Junior Data Scientist,Digital Sales Business Analyst\r\r\r\nOpportun...,Sydney NSW,"$70,000 - $90,000",NSW,Sydney,70000.0,90000.0,year,...,0.0,0.799318,0.0,0.0,0.0,0,1,0,0,0
1,Intellify,Junior data scientist/machine learning engineer,About the role : As a Business Analyst you wou...,Sydney NSW,"$80,000 - $100,000",NSW,Sydney,80000.0,100000.0,year,...,0.0,0.481811,0.0,0.0,0.0,0,1,0,0,0
2,None,Junior Data Scientist,TNT Express is one of the world's leading prov...,Sydney Central Business District NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.0,0.799318,0.0,0.0,0.0,0,1,0,0,0
3,DataRobot,Customer Facing Data Scientist,For over 15 years Quantium have combined the b...,Sydney NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.0,0,1,0,0,0
4,Freshwater Group,Data Scientist,A vacancy exists in the above unit for a highl...,Sydney NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.0,0,1,0,0,0


In [784]:
# Check the jobs df.
jobs.describe()

,Low_Salary,Hi_Salary,Salary_mean,Job_Title_analyst,Job_Title_data,Job_Title_scientist,Job_Title_business,Job_Title_senior,Job_Title_engineer,Job_Title_manager,...,Job_Title_finance,Job_Title_junior,Job_Title_financial,Job_Title_pricing,Job_Title_analytics,State_ACT,State_NSW,State_QLD,State_VIC,State_WA
count,104.000000,104.000000,104.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,...,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000
mean,103733.163462,117485.653846,110609.408654,0.141564,0.120068,0.079421,0.065005,0.054612,0.025813,0.024785,...,0.013896,0.013778,0.013643,0.013041,0.012753,0.065891,0.583333,0.106589,0.178295,0.065891
std,44653.524692,48356.641265,46006.577181,0.168572,0.212253,0.210780,0.185122,0.156650,0.110043,0.101622,...,0.096393,0.100348,0.099959,0.107165,0.081629,0.248333,0.493485,0.308890,0.383132,0.248333
min,32500.000000,34320.000000,33410.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,76181.250000,90000.000000,83127.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,95275.000000,110000.000000,103033.500000,0.129272,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,120000.000000,140000.000000,127980.500000,0.230883,0.216634,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,260000.000000,312000.000000,280800.000000,1.000000,0.807632,1.000000,0.882079,0.883968,0.865773,0.634374,...,0.945694,0.871459,0.949095,0.952783,0.757796,1.000000,1.000000,1.000000,1.000000,1.000000


### Using Vectorizers to deal the keywords in the focus columns ::  'Location', 'Job_Title' and 'Summary'

There are two types ::

1. CountVectorizer : it will just count the word frequencies and returns 1 and 0 as output.

2. TFIDFVectorizer : the value increases proportionally to count, but is offset by the frequency of the word in the corpus. - This is the IDF (inverse document frequency part). Simply, it normalizes the count i.e. the return value is like weightage of words which gives more meaning.

### Using TFID vectorizers

### 1. Job title keywords

In [785]:
#Let's start by creating some categories from the job title column


# Define a variable 'bow' (bag-of-words) for all the Job Title words.

bow = jobs['Job_Title']

In [786]:
# Fit a Job Title words with the Tfid vectorizer.

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Using the stop_words hyperparameters to exclude words which has no meaning on its own.
tvec = TfidfVectorizer(stop_words='english')

# Fit the words with the tfid vectorizer.
tvec.fit(bow)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [787]:
#Generate columns from the vectorizer
tvecdf  = pd.DataFrame(tvec.transform(bow).todense(),columns=tvec.get_feature_names())

tvecdf.transpose().sort_values(0, ascending=False).transpose().head()

,junior,scientist,data,paid,plant,planning,planner,phd,pharmacist,perth,...,executive,execution,excellence,etl,erp,equity,equities,environmental,entry,yarr
0,0.799318,0.478627,0.363327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.481811,0.288505,0.219006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.799318,0.478627,0.363327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.321165,0.243798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.796506,0.604631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [788]:
#We probably don't want to include *all* the words as features, so I will reorder the columns by how many times each word
#appears in the corpus
tvecdf = tvecdf.reindex_axis(tvecdf.sum().sort_values(ascending=False).index, axis=1)
tvecdf.head().iloc[:,:30]

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,analyst,data,scientist,business,senior,engineer,manager,reporting,commercial,research,...,project,developer,architect,science,marketing,software,insights,management,governance,risk
0,0.0,0.363327,0.478627,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.219006,0.288505,0.0,0.0,0.378886,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.363327,0.478627,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.243798,0.321165,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.604631,0.796506,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [789]:
tvecdf.shape

(516, 506)

In [790]:
# Renaming the columns.

tvecdf.rename(columns = lambda x: 'Job_Title_'+str(x), inplace=True)

In [ ]:
#Let's add the 30 most common terms to the jobs dataframe

top30 = tvecdf.iloc[:,:20]
jobs = jobs.join(top30)

In [792]:
jobs.head(2)

,Company,Job_Title,Summary,Location,Salary,State,City,Low_Salary,Hi_Salary,Salary_period,...,Job_Title_finance,Job_Title_junior,Job_Title_financial,Job_Title_pricing,Job_Title_analytics,State_ACT,State_NSW,State_QLD,State_VIC,State_WA
0,The Eclair Group,Junior Data Scientist,Digital Sales Business Analyst\r\r\r\nOpportun...,Sydney NSW,"$70,000 - $90,000",NSW,Sydney,70000.0,90000.0,year,...,0.0,0.799318,0.0,0.0,0.0,0,1,0,0,0
1,Intellify,Junior data scientist/machine learning engineer,About the role : As a Business Analyst you wou...,Sydney NSW,"$80,000 - $100,000",NSW,Sydney,80000.0,100000.0,year,...,0.0,0.481811,0.0,0.0,0.0,0,1,0,0,0


### 2. Location keywords

In [794]:
#Now to make some location-based columns
#First, note that we have a lot of variation in how particular locations are listed in the data. Some places are listed as 
#a city, such as Sydney. Others show the suburb. Almost all however, show a state. I will create state-level location
#columns from this column
jobs['Location'].value_counts().head(20)

Sydney NSW                                217
Melbourne VIC                              73
Brisbane QLD                               47
Canberra ACT                               33
Perth WA                                   30
Sydney Central Business District NSW       28
Melbourne City Centre VIC                   9
Sydney Western Suburbs NSW                  7
Brisbane Central Business District QLD      4
Rhodes NSW                                  4
Sydneyern Beaches NSW                       3
Perth Central Business District WA          3
Mascot NSW                                  3
Sydney Inner Suburbs NSW                    3
Burwood NSW                                 2
Sydney Sydney NSW                           2
Newcastle NSW                               2
Dandenong VIC                               2
Pyrmont NSW                                 2
Eight Mile Plains Mile Plains QLD           2
Name: Location, dtype: int64

In [795]:
#Make dummy columns for states
state_dummies = pd.get_dummies(jobs['State'])
state_dummies.rename(columns = lambda x: 'State_'+str(x), inplace=True)
state_dummies.sample(5)

,State_ACT,State_NSW,State_QLD,State_VIC,State_WA
267,0,0,0,1,0
238,0,1,0,0,0
112,0,1,0,0,0
130,0,0,0,0,1
254,0,0,0,1,0


In [796]:
state_dummies.columns

Index(['State_ACT', 'State_NSW', 'State_QLD', 'State_VIC', 'State_WA'], dtype='object')

In [ ]:
#Add them to the dataframe
jobs = jobs.join(state_dummies)

In [798]:
# Verify the df
jobs.head()

,Company,Job_Title,Summary,Location,Salary,State,City,Low_Salary,Hi_Salary,Salary_period,...,Job_Title_finance,Job_Title_junior,Job_Title_financial,Job_Title_pricing,Job_Title_analytics,State_ACT,State_NSW,State_QLD,State_VIC,State_WA
0,The Eclair Group,Junior Data Scientist,Digital Sales Business Analyst\r\r\r\nOpportun...,Sydney NSW,"$70,000 - $90,000",NSW,Sydney,70000.0,90000.0,year,...,0.0,0.799318,0.0,0.0,0.0,0,1,0,0,0
1,Intellify,Junior data scientist/machine learning engineer,About the role : As a Business Analyst you wou...,Sydney NSW,"$80,000 - $100,000",NSW,Sydney,80000.0,100000.0,year,...,0.0,0.481811,0.0,0.0,0.0,0,1,0,0,0
2,None,Junior Data Scientist,TNT Express is one of the world's leading prov...,Sydney Central Business District NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.0,0.799318,0.0,0.0,0.0,0,1,0,0,0
3,DataRobot,Customer Facing Data Scientist,For over 15 years Quantium have combined the b...,Sydney NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.0,0,1,0,0,0
4,Freshwater Group,Data Scientist,A vacancy exists in the above unit for a highl...,Sydney NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.0,0,1,0,0,0


In [799]:
jobs.columns

Index(['Company', 'Job_Title', 'Summary', 'Location', 'Salary', 'State',
       'City', 'Low_Salary', 'Hi_Salary', 'Salary_period', 'Salary_mean',
       'Job_Title_analyst', 'Job_Title_data', 'Job_Title_scientist',
       'Job_Title_business', 'Job_Title_senior', 'Job_Title_engineer',
       'Job_Title_manager', 'Job_Title_reporting', 'Job_Title_commercial',
       'Job_Title_research', 'Job_Title_technical', 'Job_Title_consultant',
       'Job_Title_digital', 'Job_Title_lead', 'Job_Title_st',
       'Job_Title_finance', 'Job_Title_junior', 'Job_Title_financial',
       'Job_Title_pricing', 'Job_Title_analytics', 'State_ACT', 'State_NSW',
       'State_QLD', 'State_VIC', 'State_WA'],
      dtype='object')

### 3. Summary keywords

In [800]:
#First clean up unwanted characters such as newline and - characters
unwanted = '\r\n-'
removechars = str.maketrans(' ',' ',unwanted)
jobs['Summary'] = jobs['Summary'].apply(lambda x: x.translate(removechars))

In [801]:
#I will look for important summary keywords using a Tf-idf vectorizer
from sklearn.feature_extraction import text
from nltk.stem import PorterStemmer, SnowballStemmer

# I've elected to do stemming before extracting words from the summary since it is more likely 
#that different variants of the same word will appear in the summary than the job title.

In [802]:
#First I'll do some stemming to take care of redundant word endings (e.g. 'experience' vs 'experienced')
stemmer = SnowballStemmer('english')
jobs['stemmed_summary'] = [[stemmer.stem(word) for word in sentence.split(" ")] for sentence in jobs['Summary']]

In [803]:
for i in jobs['stemmed_summary'].index:
    jobs['stemmed_summary'][i] = str(jobs['stemmed_summary'][i])

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [804]:
stop_words = text.ENGLISH_STOP_WORDS.union(['work','busi','role','look','build','strong','understand','grow','ensur',
                                           'join','key','requir','year','help','www','weeks','deal','mid','exist','datayou'])
tvec = TfidfVectorizer(stop_words=stop_words)
tvec.fit(jobs['stemmed_summary'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=frozenset({'because', 'beside', 'due', 'we', 'another', 'behind', 'but', 'whose', 'even', 'mill', 'ten', 'became', 'seemed', 'yours', 'nobody', 'becoming', 'several', 'off', 'across', 'him', 'often', 'never', 'how', 'forty', 'four', 'further', 'cry', 'together', 'through', 'anything', 'mo...ween', 'enough', 'someone', 'side', 'nor', 'interest', 'their', 'around', 'toward', 'below', 'who'}),
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [805]:
tvecdf  = pd.DataFrame(tvec.transform(jobs['stemmed_summary']).todense(),
                   columns=tvec.get_feature_names())

tvecdf.transpose().sort_values(0, ascending=False).head(10).transpose()
#Order columns based on the total sum of tf-idf scores
tvecdf = tvecdf.reindex_axis(tvecdf.mean().sort_values(ascending=False).index, axis=1)

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  


In [806]:
tvecdf.shape

(516, 11315)

In [807]:
#Some of these seem like sensible words to look at (like, business, analytics, research, customer etc.). While others
#are likely to be uninformative (e.g work, role, ability) I will look through the top 30 and manually drop columns that seem
#uninformative.
tvecdf.head().iloc[:,:30]

,data,experi,team,analyt,manag,develop,project,custom,analyst,process,...,appli,product,technic,financi,solut,stakehold,insight,market,learn,scienc
0,0.014715,0.077239,0.000000,0.022185,0.061174,0.000000,0.023471,0.051507,0.056531,0.069611,...,0.000000,0.026881,0.000000,0.000000,0.074849,0.024304,0.000000,0.000000,0.0,0.000000
1,0.030436,0.079876,0.017799,0.045884,0.063263,0.038422,0.072816,0.026632,0.058461,0.023996,...,0.000000,0.083395,0.050626,0.000000,0.025801,0.075402,0.000000,0.000000,0.0,0.000000
2,0.000000,0.034800,0.038772,0.000000,0.114840,0.062772,0.026437,0.000000,0.042450,0.104543,...,0.023971,0.000000,0.000000,0.067506,0.000000,0.109502,0.031831,0.000000,0.0,0.000000
3,0.064494,0.009672,0.086209,0.013890,0.051069,0.023262,0.029391,0.032249,0.035395,0.000000,...,0.013325,0.218792,0.000000,0.000000,0.031242,0.015217,0.053083,0.037853,0.0,0.020044
4,0.030105,0.015802,0.052816,0.000000,0.062575,0.057006,0.240081,0.000000,0.038550,0.023735,...,0.000000,0.027496,0.000000,0.000000,0.025521,0.000000,0.000000,0.000000,0.0,0.032747


In [808]:
top1000 = tvecdf.iloc[:,:1000]
top1000.columns

Index(['data', 'experi', 'team', 'analyt', 'manag', 'develop', 'project',
       'custom', 'analyst', 'process',
       ...
       'sure', 'fintech', 'configur', 'commenc', 'loyalti',
       'environmentstrong', 'valuabl', 'game', 'institut', 'experts'],
      dtype='object', length=1000)

In [809]:
#Make a full table of keywords
summ_keywords = top1000#.join(extra)
#Ad the prefix kw_ to the column name to indicate that these are summary keywords
summ_keywords.rename(columns = lambda x: 'kw_'+str(x), inplace=True)
summ_keywords.head()

D:\ANACONDA\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,kw_data,kw_experi,kw_team,kw_analyt,kw_manag,kw_develop,kw_project,kw_custom,kw_analyst,kw_process,...,kw_sure,kw_fintech,kw_configur,kw_commenc,kw_loyalti,kw_environmentstrong,kw_valuabl,kw_game,kw_institut,kw_experts
0,0.014715,0.077239,0.000000,0.022185,0.061174,0.000000,0.023471,0.051507,0.056531,0.069611,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0655,0.0,0.000000
1,0.030436,0.079876,0.017799,0.045884,0.063263,0.038422,0.072816,0.026632,0.058461,0.023996,...,0.120944,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000
2,0.000000,0.034800,0.038772,0.000000,0.114840,0.062772,0.026437,0.000000,0.042450,0.104543,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000
3,0.064494,0.009672,0.086209,0.013890,0.051069,0.023262,0.029391,0.032249,0.035395,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.077067,0.0000,0.0,0.035792
4,0.030105,0.015802,0.052816,0.000000,0.062575,0.057006,0.240081,0.000000,0.038550,0.023735,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000


In [810]:
#Finally, add these keyword columns to the main dataframe
jobs = jobs.join(summ_keywords)

In [811]:
jobs.shape

(516, 1037)

In [812]:
jobs.head()

,Company,Job_Title,Summary,Location,Salary,State,City,Low_Salary,Hi_Salary,Salary_period,...,kw_sure,kw_fintech,kw_configur,kw_commenc,kw_loyalti,kw_environmentstrong,kw_valuabl,kw_game,kw_institut,kw_experts
0,The Eclair Group,Junior Data Scientist,Digital Sales Business AnalystOpportunity to c...,Sydney NSW,"$70,000 - $90,000",NSW,Sydney,70000.0,90000.0,year,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0655,0.0,0.000000
1,Intellify,Junior data scientist/machine learning engineer,About the role : As a Business Analyst you wou...,Sydney NSW,"$80,000 - $100,000",NSW,Sydney,80000.0,100000.0,year,...,0.120944,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000
2,None,Junior Data Scientist,TNT Express is one of the world's leading prov...,Sydney Central Business District NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000
3,DataRobot,Customer Facing Data Scientist,For over 15 years Quantium have combined the b...,Sydney NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.077067,0.0000,0.0,0.035792
4,Freshwater Group,Data Scientist,A vacancy exists in the above unit for a highl...,Sydney NSW,None,NSW,Sydney,NaN,NaN,NaN,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000


### Creating Skills

In [813]:
jobs['skill_sql'] = [1 if 'sql' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_python'] = [1 if 'python' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_spark'] = [1 if 'spark' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_hadoop'] = [1 if 'hadoop' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_tableau'] = [1 if 'tableau' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_r'] = [1 if 'r' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_excel'] = [1 if 'excel' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_sas'] = [1 if 'sas' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_spss'] = [1 if 'spss' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_aws'] = [1 if 'aws' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_kafka'] = [1 if 'kafka' in x else 0 for x in jobs['stemmed_summary']]

### NEXT PART in 4.3_Feature_Extraction_&_Modelling

In [814]:
#Now that we've made feature columns, we have some remaining columns that won't be useful for our modelling.
#These includes the remaining salary info, the full summary, the stemmed summary and the state and original location column.
#I will keep job title and company for indexing purposes.
dropcols = ['Location','Salary','Summary','State','stemmed_summary']

In [815]:
jobs = jobs.drop(dropcols, axis=1)

In [816]:
jobs.head()

,Company,Job_Title,City,Low_Salary,Hi_Salary,Salary_period,Salary_mean,Job_Title_analyst,Job_Title_data,Job_Title_scientist,...,skill_python,skill_spark,skill_hadoop,skill_tableau,skill_r,skill_excel,skill_sas,skill_spss,skill_aws,skill_kafka
0,The Eclair Group,Junior Data Scientist,Sydney,70000.0,90000.0,year,80000.0,0.0,0.363327,0.478627,...,0,0,0,0,1,0,0,0,0,0
1,Intellify,Junior data scientist/machine learning engineer,Sydney,80000.0,100000.0,year,90000.0,0.0,0.219006,0.288505,...,0,0,0,0,1,1,0,0,0,0
2,None,Junior Data Scientist,Sydney,NaN,NaN,NaN,NaN,0.0,0.363327,0.478627,...,0,0,0,0,1,1,0,0,0,0
3,DataRobot,Customer Facing Data Scientist,Sydney,NaN,NaN,NaN,NaN,0.0,0.243798,0.321165,...,0,0,1,0,1,1,0,0,0,0
4,Freshwater Group,Data Scientist,Sydney,NaN,NaN,NaN,NaN,0.0,0.604631,0.796506,...,0,0,0,0,1,1,0,0,0,0


In [817]:
# Saving the file after data clean-up:

jobs.to_csv('./jobs_part2_cleanup.csv',index=False, encoding='utf-8')